In [14]:
import pandas as pd    # to load dataset
import numpy as np     # for mathematic equation
from nltk.corpus import stopwords   # to get collection of stopwords
from sklearn.model_selection import train_test_split       # for splitting dataset
from tensorflow.keras.preprocessing.text import Tokenizer  # to encode text to int
from tensorflow.keras.preprocessing.sequence import pad_sequences   # to do padding or truncating
from tensorflow.keras.models import Sequential     # the model
from tensorflow.keras.layers import Embedding, LSTM, Dense # layers of the architecture
from tensorflow.keras.callbacks import ModelCheckpoint   # save model
from tensorflow.keras.models import load_model   # load saved model

In [15]:
data = pd.read_csv('IMDB Dataset.csv')

print(data)


                                                  review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]


In [16]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
english_stops = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
def load_dataset():
    df = pd.read_csv('IMDB Dataset.csv')
    x_data = df['review']       # Reviews/Input
    y_data = df['sentiment']    # Sentiment/Output

    # PRE-PROCESS REVIEW
    x_data = x_data.replace({'<.*?>': ''}, regex = True)          # remove html tag
    x_data = x_data.replace({'[^A-Za-z]': ' '}, regex = True)     # remove non alphabet
    x_data = x_data.apply(lambda review: [w for w in review.split() if w not in english_stops])  # remove stop words
    x_data = x_data.apply(lambda review: [w.lower() for w in review])   # lower case

    # ENCODE SENTIMENT -> 0 & 1
    y_data = y_data.replace('positive', 1)
    y_data = y_data.replace('negative', 0)

    return x_data, y_data

x_data, y_data = load_dataset()

print('Reviews')
print(x_data, '\n')
print('Sentiment')
print(y_data)

Reviews
0        [one, reviewers, mentioned, watching, oz, epis...
1        [a, wonderful, little, production, the, filmin...
2        [i, thought, wonderful, way, spend, time, hot,...
3        [basically, family, little, boy, jake, thinks,...
4        [petter, mattei, love, time, money, visually, ...
                               ...                        
49995    [i, thought, movie, right, good, job, it, crea...
49996    [bad, plot, bad, dialogue, bad, acting, idioti...
49997    [i, catholic, taught, parochial, elementary, s...
49998    [i, going, disagree, previous, comment, side, ...
49999    [no, one, expects, star, trek, movies, high, a...
Name: review, Length: 50000, dtype: object 

Sentiment
0        1
1        1
2        1
3        0
4        1
        ..
49995    1
49996    0
49997    0
49998    0
49999    0
Name: sentiment, Length: 50000, dtype: int64


/tmp/ipython-input-4067306172.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_data = y_data.replace('negative', 0)


In [18]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2)

print('Train Set')
print(x_train, '\n')
print(x_test, '\n')
print('Test Set')
print(y_train, '\n')
print(y_test)


Train Set
1566     [had, ned, kelly, born, later, probably, would...
43775    [you, see, believe, directors, alastair, fothe...
11434    [badly, made, dreadful, acting, ending, direct...
19305    [without, question, film, one, greatest, cinem...
43877    [i, surprised, discover, michael, moore, bill,...
                               ...                        
22636    [saw, movie, early, preview, i, cannot, stress...
38768    [it, interesting, see, accurate, writing, geek...
3816     [cave, dwellers, the, blade, mater, whatever, ...
17345    [very, good, precoder, starring, dick, barthel...
19288    [this, movie, i, seen, prince, matter, and, i,...
Name: review, Length: 40000, dtype: object 

41159    [the, year, ernesto, che, guevara, cuban, citi...
49401    [many, us, went, high, school, probably, made,...
25306    [outstanding, film, paul, lukas, giving, oscar...
14061    [this, snuff, movie, i, shocked, even, conside...
17303    [flash, gordon, first, rate, serial, i, know, ...
 

In [19]:
def get_max_length():
    review_length = []
    for review in x_train:
        review_length.append(len(review))

    return int(np.ceil(np.mean(review_length)))

In [20]:
# ENCODE REVIEW
token = Tokenizer(lower=False)    # no need lower, because already lowered the data in load_data()
token.fit_on_texts(x_train)
x_train = token.texts_to_sequences(x_train)
x_test = token.texts_to_sequences(x_test)

max_length = get_max_length()

x_train = pad_sequences(x_train, maxlen=max_length, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=max_length, padding='post', truncating='post')

total_words = len(token.word_index) + 1   # add 1 because of 0 padding

print('Encoded X Train\n', x_train, '\n')
print('Encoded X Test\n', x_test, '\n')
print('Maximum review length: ', max_length)

Encoded X Train
 [[ 2841  3702  1446 ...  9916   147    62]
 [   96    13   165 ...     0     0     0]
 [  849    24  2005 ...     0     0     0]
 ...
 [ 3006 12630     2 ...     0     0     0]
 [  793     9 92320 ...     0     0     0]
 [    8     3     1 ...     0     0     0]] 

Encoded X Test
 [[    2   198 17943 ...     0     0     0]
 [   37    88   336 ...   352    27  2588]
 [ 1309     4   717 ...     0     0     0]
 ...
 [   49   132  6264 ...     0     0     0]
 [    8     5   156 ...     0     0     0]
 [    1    18   360 ...     0     0     0]] 

Maximum review length:  131


In [23]:
# ARCHITECTURE
EMBED_DIM = 32
LSTM_OUT = 64

model = Sequential()
model.add(Embedding(total_words, EMBED_DIM, input_length = max_length))
model.add(LSTM(LSTM_OUT))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.build(input_shape=(None, max_length))
print(model.summary())

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ (None, 131, 32)        │     2,954,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 64)             │        24,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,979,201 (11.36 MB)

 Trainable params: 2,979,201 (11.36 MB)

 Non-trainable params: 0 (0.00 B)

None


In [24]:
checkpoint = ModelCheckpoint(
    'models/LSTM.h5',
    monitor='accuracy',
    save_best_only=True,
    verbose=1
)

In [25]:
model.fit(x_train, y_train, batch_size = 128, epochs = 5, callbacks=[checkpoint])

Epoch 1/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step - accuracy: 0.5682 - loss: 0.6546
Epoch 1: accuracy improved from -inf to 0.66067, saving model to models/LSTM.h5


313/313 ━━━━━━━━━━━━━━━━━━━━ 65s 199ms/step - accuracy: 0.5685 - loss: 0.6545
Epoch 2/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step - accuracy: 0.6943 - loss: 0.5737
Epoch 2: accuracy improved from 0.66067 to 0.75100, saving model to models/LSTM.h5


313/313 ━━━━━━━━━━━━━━━━━━━━ 61s 196ms/step - accuracy: 0.6945 - loss: 0.5736
Epoch 3/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step - accuracy: 0.8625 - loss: 0.3849
Epoch 3: accuracy improved from 0.75100 to 0.83293, saving model to models/LSTM.h5


313/313 ━━━━━━━━━━━━━━━━━━━━ 62s 197ms/step - accuracy: 0.8624 - loss: 0.3850
Epoch 4/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step - accuracy: 0.7541 - loss: 0.5482
Epoch 4: accuracy did not improve from 0.83293
313/313 ━━━━━━━━━━━━━━━━━━━━ 62s 199ms/step - accuracy: 0.7540 - loss: 0.5483
Epoch 5/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step - accuracy: 0.6196 - loss: 0.6465
Epoch 5: accuracy did not improve from 0.83293
313/313 ━━━━━━━━━━━━━━━━━━━━ 81s 196ms/step - accuracy: 0.6196 - loss: 0.6465


In [27]:
y_pred_prob = model.predict(x_test, batch_size = 128)
y_pred = (y_pred_prob > 0.5).astype(int)


true = 0
for i, y in enumerate(y_test):
    if y == y_pred[i]:
        true += 1

print('Correct Prediction: {}'.format(true))
print('Wrong Prediction: {}'.format(len(y_pred) - true))
print('Accuracy: {}'.format(true/len(y_pred)*100))

79/79 ━━━━━━━━━━━━━━━━━━━━ 6s 76ms/step
Correct Prediction: 6168
Wrong Prediction: 3832
Accuracy: 61.68


In [28]:
loaded_model = load_model('models/LSTM.h5')

In [29]:
review = str(input('Movie Review: '))

Movie Review: Nothing was typical about this. Everything was beautifully done in this movie, the story, the flow, the scenario, everything. I highly recommend it for mystery lovers, for anyone who wants to watch a good movie!


In [32]:
import re
# Pre-process input
regex = re.compile(r'[^a-zA-Z\s]')
review = regex.sub('', review)
# print('Cleaned: ', review)

words = review.split(' ')
filtered = [w for w in words if w not in english_stops]
filtered = ' '.join(filtered)
filtered = [filtered.lower()]

# print('Filtered: ', filtered)

tokenize_words = token.texts_to_sequences(filtered)
tokenize_words = pad_sequences(tokenize_words, maxlen=max_length, padding='post', truncating='post')
print(tokenize_words)

[[  76  692  176 1197  126    3   14 2692 2634  176    1  452  282  701
  1707  153  395   33    9    3    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0]]


In [33]:
result = loaded_model.predict(tokenize_words)
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 311ms/step
[[0.7367924]]


In [34]:
if result >= 0.7:
    print('positive')
else:
    print('negative')

positive
